# Курсовой проект по циклу "Машинное обучение в бизнесе"

## Прогнозирование рака легких 

датасет: https://www.kaggle.com/datasets/nancyalaswad90/lung-cancer

legend:

1. Gender: M(male), F(female) 
2. Age: Age of the patient 
3. Smoking: YES=2 , NO=1. 
4. Yellow fingers: YES=2 , NO=1. 
5. Anxiety: YES=2 , NO=1. 
6. Peer_pressure: YES=2 , NO=1. 
7. Chronic Disease: YES=2 , NO=1. 
8. Fatigue: YES=2 , NO=1. 
9. Allergy: YES=2 , NO=1. 
10. Wheezing: YES=2 , NO=1. 
11. Alcohol: YES=2 , NO=1. 
12. Coughing: YES=2 , NO=1. 
13. Shortness of Breath: YES=2 , NO=1. 
14. Swallowing Difficulty: YES=2 , NO=1. 
15. Chest pain: YES=2 , NO=1. 
16. Lung Cancer: YES , NO. - tareget

имортируем необходимые бибилотеки и модули

In [1]:
import pandas as pd
import numpy as np

import dill


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV 

# from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix, log_loss, roc_curve

import itertools

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = pd.read_csv('survey_lung_cancer.csv')
dataset.head(5)

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 309 non-null    object
 1   AGE                    309 non-null    int64 
 2   SMOKING                309 non-null    int64 
 3   YELLOW_FINGERS         309 non-null    int64 
 4   ANXIETY                309 non-null    int64 
 5   PEER_PRESSURE          309 non-null    int64 
 6   CHRONIC DISEASE        309 non-null    int64 
 7   FATIGUE                309 non-null    int64 
 8   ALLERGY                309 non-null    int64 
 9   WHEEZING               309 non-null    int64 
 10  ALCOHOL CONSUMING      309 non-null    int64 
 11  COUGHING               309 non-null    int64 
 12  SHORTNESS OF BREATH    309 non-null    int64 
 13  SWALLOWING DIFFICULTY  309 non-null    int64 
 14  CHEST PAIN             309 non-null    int64 
 15  LUNG_CANCER            

проверим наличие пропусков

In [4]:
dataset.isnull().sum()

GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL CONSUMING        0
COUGHING                 0
SHORTNESS OF BREATH      0
SWALLOWING DIFFICULTY    0
CHEST PAIN               0
LUNG_CANCER              0
dtype: int64

соотношение целевой перменной

In [5]:
dataset['LUNG_CANCER'].value_counts()

YES    270
NO      39
Name: LUNG_CANCER, dtype: int64

In [6]:
numeric_cols=dataset.select_dtypes(include=np.number).columns.tolist()
categorial_cols=dataset.select_dtypes(object).columns.tolist()

print(f'Вывод: \nДатасет содержит 309 строк (наблюдений), 16 столбцов, в том числе: 15 признаков и 1 - целевая переменная.\n\n' 
      f'Тип данных int64 встречается у {len(numeric_cols)} столбцов: {numeric_cols},\n\nobject - у {len(categorial_cols)}: {categorial_cols}.\n\n'
      f'Пропуски в датасете отсутствуют.\n\n'
     f'Соотношение классов целевой переменной не сбалансировано')

Вывод: 
Датасет содержит 309 строк (наблюдений), 16 столбцов, в том числе: 15 признаков и 1 - целевая переменная.

Тип данных int64 встречается у 14 столбцов: ['AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING', 'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH', 'SWALLOWING DIFFICULTY', 'CHEST PAIN'],

object - у 2: ['GENDER', 'LUNG_CANCER'].

Пропуски в датасете отсутствуют.

Соотношение классов целевой переменной не сбалансировано


## EDA (exploratory data analysis)

Категориальные признаки

In [7]:
print(f'{len(categorial_cols)} категориальных признака\n{categorial_cols}\n')

for i in categorial_cols:
    unique_no = dataset[i].nunique()
    unique_name = dataset[i].unique().tolist()
    print(f'{i}, имеет {unique_no} уникальных значения:\n{unique_name}\n')

2 категориальных признака
['GENDER', 'LUNG_CANCER']

GENDER, имеет 2 уникальных значения:
['M', 'F']

LUNG_CANCER, имеет 2 уникальных значения:
['YES', 'NO']



## Feature engineering

переименуем столбцы

In [8]:
dataset.columns = ['GENDER', 'AGE', 'SMOKING','YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE', 'CHRONIC_DISEASE', 'FATIGUE',
                   'ALLERGY', 'WHEEZING', 'ALCOHOL_CONSUMING', 'COUGHING', 'SHORTNESS_OF_BREATH', 
                   'SWALLOWING_DIFFICULTY', 'CHEST_PAIN', 'LUNG_CANCER']

изменим значения полей, содержащих категориальные признаки на бинарные (исходные: YES = 2 и NO = 1 на YES = 1 и NO = 0)

поля: 'SMOKING','YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE', 'ALLERGY',
'WHEEZING', 'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH', 'SWALLOWING DIFFICULTY', 'CHEST PAIN' 

In [9]:
list_feat_names = ['SMOKING','YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE', 'CHRONIC_DISEASE', 'FATIGUE',
                   'ALLERGY', 'WHEEZING', 'ALCOHOL_CONSUMING', 'COUGHING', 'SHORTNESS_OF_BREATH', 
                   'SWALLOWING_DIFFICULTY', 'CHEST_PAIN']


for name in list_feat_names:
    dataset[name] = dataset[name].replace([2, 1],[1, 0])

In [10]:
dataset['GENDER'] = dataset['GENDER'].map({'M': 1, 'F':0})

In [11]:
dataset['LUNG_CANCER'] = dataset['LUNG_CANCER'].map({'YES': 1, 'NO':0})

In [12]:
row_40 = dataset[dataset['AGE'] <= 40 ].index
dataset = dataset.drop(dataset.index[row_40])

In [13]:
row_80 = dataset[dataset['AGE'] > 80].index
dataset = dataset.drop(dataset.index[row_80])

In [14]:
dataset.head(3)

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,1,69,0,1,1,0,0,1,0,1,1,1,1,1,1,1
1,1,74,1,0,0,0,1,1,1,0,0,0,1,1,1,1
2,0,59,0,0,0,1,0,1,0,1,0,1,1,0,1,0


In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   GENDER                 303 non-null    int64
 1   AGE                    303 non-null    int64
 2   SMOKING                303 non-null    int64
 3   YELLOW_FINGERS         303 non-null    int64
 4   ANXIETY                303 non-null    int64
 5   PEER_PRESSURE          303 non-null    int64
 6   CHRONIC_DISEASE        303 non-null    int64
 7   FATIGUE                303 non-null    int64
 8   ALLERGY                303 non-null    int64
 9   WHEEZING               303 non-null    int64
 10  ALCOHOL_CONSUMING      303 non-null    int64
 11  COUGHING               303 non-null    int64
 12  SHORTNESS_OF_BREATH    303 non-null    int64
 13  SWALLOWING_DIFFICULTY  303 non-null    int64
 14  CHEST_PAIN             303 non-null    int64
 15  LUNG_CANCER            303 non-null    i

разделим датасет на тренировочную и тестовую выборку 

In [16]:
features = ['GENDER', 'AGE', 'SMOKING','YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE', 'CHRONIC_DISEASE', 'FATIGUE',
                   'ALLERGY', 'WHEEZING', 'ALCOHOL_CONSUMING', 'COUGHING', 'SHORTNESS_OF_BREATH', 
                   'SWALLOWING_DIFFICULTY', 'CHEST_PAIN']
target = 'LUNG_CANCER'


X_train, X_test, y_train, y_test = train_test_split(dataset[features], dataset[target], test_size=0.33, random_state=42,
                                                   stratify=dataset[target])

#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [17]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='GENDER')),
                ('ohe', OHEEncoder(key='GENDER'))
            ])

gender.fit_transform(X_train)

,GENDER_0,GENDER_1
147,1,0
214,1,0
79,1,0
210,0,1
109,0,1
...,...,...
274,0,1
95,0,1
3,0,1
227,0,1


In [18]:
number_features = Pipeline([
                ('selector', ColumnSelector(key=['AGE', 'SMOKING','YELLOW_FINGERS', 
                                                 'ANXIETY', 'PEER_PRESSURE', 
                                                 'CHRONIC_DISEASE', 'FATIGUE',
                                                 'ALLERGY', 'WHEEZING', 
                                                 'ALCOHOL_CONSUMING', 
                                                 'COUGHING', 'SHORTNESS_OF_BREATH',
                                                 'SWALLOWING_DIFFICULTY', 'CHEST_PAIN']))])
feats = FeatureUnion([('continuos_features', number_features),
                      ('gender', gender)]) 
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

array([[56,  0,  0, ...,  0,  1,  0],
       [61,  1,  1, ...,  1,  1,  0],
       [73,  1,  1, ...,  1,  1,  0],
       ...,
       [63,  1,  1, ...,  1,  0,  1],
       [71,  0,  1, ...,  0,  0,  1],
       [67,  1,  0, ...,  1,  0,  1]], dtype=int64)

In [19]:
%%time

pipeline = Pipeline([
    ('features', feats),
#     ('classifier', RandomForestClassifier(random_state = 42)),
    ('classifier', GradientBoostingClassifier(random_state = 42)),    
])

CPU times: total: 0 ns
Wall time: 0 ns


In [20]:
#запустим кросс-валидацию
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
pipeline.fit(X_train, y_train)
y_score = pipeline.predict_proba(X_test)[:, 1]



CV score is 0.8958333333333333+-0.1559828984233559


Посмотрим, как выглядит наш pipeline

In [21]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('continuos_features',
                                  Pipeline(steps=[('selector',
                                                   ColumnSelector(key=['AGE',
                                                                       'SMOKING',
                                                                       'YELLOW_FINGERS',
                                                                       'ANXIETY',
                                                                       'PEER_PRESSURE',
                                                                       'CHRONIC_DISEASE',
                                                                       'FATIGUE',
                                                                       'ALLERGY',
                                                                       'WHEEZING',
                                                                       'ALCOHOL_CONSUMING',
                   

Сохраним модель (пайплайн)

In [22]:
with open("GBC_model.dill", "wb") as f:
    dill.dump(pipeline, f)